Bibliothèques utiles :

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from cfd_common import *
from math import *
import scipy.interpolate
import os

In [2]:
%matplotlib qt

In [3]:
plt.ion()

In [4]:
path_prefix = 'avalanche_monster_a0.2_u1.8_g10_Re1000_660x120_1'
if not os.path.exists(path_prefix):
    os.mkdir(path_prefix)

Choix du domaine et des paramètres de la discrétisation du problème :

In [5]:
##### Domain #####

# Domain size
Lx = 6.
Ly = 1.

# Physical grid size
Nx_phys = 660
Ny_phys = 120
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys

# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

Initialisation des champs de vitesse / champ de pression et champ de densité :

On note $u,v$ les composantes horizontale, verticales du champ de vitesse. On les initialise à :...

In [6]:
##### Fields #####

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
v = np.zeros((Nx,Ny))

vrand_ampli = 0
v += vrand_ampli * ( 1-2*np.random.rand(Nx,Ny) ) # random vertical velocity to speed up the instability

# Pressure field phi[i,j] and its gradient (dx_phi,dy_phi)
phi = np.zeros((Nx,Ny))
dx_phi = np.zeros((Nx,Ny))
dy_phi = np.zeros((Nx,Ny))
phi0_left = np.zeros(Ny_phys)
phi0_right = np.zeros(Ny_phys)
phi0_top = np.zeros(Nx_phys)

# Density field
ρ0 = 1
ρ1 = 10
ρ = ρ0*np.ones((Nx,Ny))

#ρ[:, 4*Ny//10 : 8*Ny//10] = 1
#ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.5, 5, 1)
#ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.3, 1, ρ[1:-1,1:-1])

#ρ[1:-1,1:-1] = np.transpose(np.where(((xx-Lx/2)**2+(yy-0.15*Ly)**2)<0.2**2, 1, 5))

surface0 = 0.3
surface = surface0 + 0.05 * np.random.normal(size=Nx_phys)
ρ_init = np.where( (yy < np.stack([surface]*Ny_phys,0)) & (xx < 2) & (xx > 0.05), ρ1, ρ0 )
ρ_init = np.where( (yy < 0.15) & (xx < 2) & (xx > 1.7), ρ0, ρ_init )
ρ[1:-1,1:-1] = np.transpose( ρ_init )

u = 0.2*(ρ-ρ0)

On impose les conditions aux bords (sur les GhostPoints) :

In [7]:
def VelocityGhostPoints(u,v):
    ### left
    u[0, :] = u[1, :]
    v[0, :] = v[1, :]
    ### right    
    u[-1, :] = u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = -u[:,  1]  # no slip
    v[:,  0] = -v[:,  1]  # imperméabilité
    ### top     
    u[:, -1] = u[:, -2]
    v[:, -1] = v[:, -2]

def DensityGhostPoints(ρ):
    # ghost points for density : Neumann ∂ρ=0 boundary condition
#    ρ[ 0, 1:-1] = np.where( y > surface0, ρ0, ρ1 ) # left
    ρ[ 0, :] = ρ0
    ρ[-1, :] = ρ0 #ρ[-2, :] # right
    ρ[:,  0] = ρ[:, 1] # bottom
    ρ[:, -1] = ρ0 #ρ[:, -2] # top  

Ensuite vient la définition de notre schéma :

In [8]:
###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAP,LAP_solver = FD_2D_Laplacian_matrix (Nx_phys, Ny_phys, Δx, Δy, BCdir_left=True, BCdir_right=True, BCdir_top=True, BCdir_bot=False)

Puis la boucle principale de calcul :

In [9]:
### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.00005

t = 0. # total time

# parameters
Re = 1000 # Reynolds number
g = 10
α = 0.2*pi

# inertia of the fluid behind the boundary, from 1 (full inwards self-sustaining advection "free flow")
#  to 0 (no inwards advection, "free stokes flow")
bnd_inertia = 0.5

# how much do we correct for diffusion in semi-lag advection at the price of dispersion
ρ_semilag2_coeff = 0.8
u_semilag2_coeff = 0.9

niter = 0
disp_modulo = 60

update_side_pressures_modulo = 10

In [10]:
#data = np.load("bubble-test-200x200-t2.40237-niter240237.npz")
#u = data['u']
#v = data['v']
#ρ = data['rho']
#t = 2.40237
#niter = 240237
# np.savez("bubble-test-200x200-.npz", u=u, v=v, rho=ρ)

In [11]:
## Check mass
def mass_check(rho):
    return Δx*Δy*np.sum(rho)

mass_check(ρ)

10.917121212121213

In [12]:
def compute_hydrostatic_pressure (x0, y0, npoints=100):
    z = np.linspace(0, (Ly-y0+x0*sin(α))/(cos(α)+sin(α)**2), npoints)
    x_int = x0 - z * sin(α)
    y_int = y0 + z * cos(α)
    Δl = sqrt( (y_int[1]-y_int[0])**2 + (x_int[1]-x_int[0])**2 )
    ρ_f = scipy.interpolate.RectBivariateSpline(x, y, ρ[1:-1,1:-1])
    def ρ_f_extra (x, y):
        if x > Lx or x < 0 or y > Ly or y < 0:
            return ρ0
        else:
            return ρ_f(x,y)
    ρ_f_extra = np.vectorize(ρ_f_extra)
    ρ_sample = ρ_f_extra(x_int, y_int)
    return g * Δl*np.sum(ρ_sample)

def compute_hydrostatic_pressure (x0, y0, npoints=100):
    return g * ρ0 * ( (Ly-y0) * cos(α) + x0 * sin(α) )

phi[1:-1,1:-1] = np.transpose( compute_hydrostatic_pressure(xx,yy) )

In [13]:
fig = plt.figure(1, figsize=(14,7))
fig2 = plt.figure(2, figsize=(14,4))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)

def do_plot (save_image):
    fig.clear()
    (ax1,ax2) = fig.subplots(nrows=2, ncols=1, sharex=True)

    def plot_vel (ax, modx=4, mody=4, vmax=100):
        if modx is None:
            modx = max(1,int(round(Nx_phys/50)))
        if mody is None:
            mody = max(1,int(round(Ny_phys/50)))
        ax.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(u[1:-1,1:-1])[::modx,::mody], np.transpose(v[1:-1,1:-1])[::modx,::mody], scale=vmax*5, minlength=0.001, headwidth=4, headlength=6, headaxislength=5, width=0.0006)
        ax.set_aspect('equal', adjustable='box')

    base_text = r" at $t={:.3f}$. Free-flow B.C., no-slip B.C. bottom. $Re={}$, SemiLag2({}) for $\vec{{u}}$, ${}\times{}$ grid.{} SemiLag2({}) for $\rho$ (min 0.2), total mass : {:.2f}. $\Delta t=${:.0e}. Slope $\alpha={:.0f}$°, $g={}$. Boundary intertia : {}".format(t,Re,u_semilag2_coeff,Nx_phys,Ny_phys,"\n",ρ_semilag2_coeff,mass_check(ρ),Δt,α/np.pi*180,g,bnd_inertia)
    fig.suptitle("Velocity, Density and Pressure fields"+base_text)

    ax1.imshow(np.transpose(ρ[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu, vmin=ρ0-1, vmax=ρ1+1)
    plot_vel(ax1)
    
    im = ax2.imshow(np.transpose(phi[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap='plasma', vmin=-1, vmax=ρ0*g*5)
    fig.colorbar(im)
    plot_vel(ax2, modx=6, mody=6)

#        my_cmap = plt.cm.gnuplot2_r(np.arange(plt.cm.gnuplot2_r.N))
#        my_cmap = 0.5 * (1+my_cmap)
#        my_cmap = matplotlib.colors.ListedColormap(my_cmap)
#        ax1.imshow(np.transpose(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2)), origin='lower', extent=(0,Lx,0,Ly), cmap=my_cmap, vmin=0, vmax=2)
#        plot_vel(ax1)


    fig.tight_layout()
    if save_image and niter != 0:
        fig.savefig(path_prefix+'/{:04d}.png'.format(niter//disp_modulo))
    fig.canvas.draw_idle()
    
    fig2.clear()
    ax = fig2.subplots(nrows=1, ncols=1)
    ax.contour(xx, yy, np.transpose(ρ[1:-1,1:-1]), levels=[ 0.2*ρ1+0.8*ρ0, 0.5*ρ1+0.5*ρ0, 0.8*ρ1+0.1*ρ0 ], colors=['grey','black','grey'], linewidths=[1,2,1], linestyles=['dotted','solid','dotted'])
    plot_vel(ax, modx=6, mody=6, vmax=80)
    fig2.tight_layout()
    fig2.suptitle("Velocity fields and Density contour"+base_text)
    if save_image and niter != 0:
        fig2.savefig(path_prefix+'/{:04d}.ctr.png'.format(niter//disp_modulo))
    fig2.canvas.draw_idle()
    
    mpl_pause_background(0.00001)

for k in range(5):
    do_plot(False)

In [ ]:
while loop_continue:

    c = 2*bnd_inertia-1
    ### left
    u[0, :] = c * u[1, :]
    v[0, :] = c * v[1, :]
    ### right    
    u[-1, :] = c * u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = c * v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = -u[:,  1]  # no slip
    v[:,  0] = -v[:,  1]  # imperméabilité
    ### top     
    u[:, -1] = c * u[:, -2]
    v[:, -1] = c * v[:, -2]
    
    ###### Advection semi-lagrangienne
    if u_semilag2_coeff is None:
        adv_u = SemiLag(u,v, u, Δx,Δy,Δt)
        adv_v = SemiLag(u,v, v, Δx,Δy,Δt)
    else:
        adv_u = SemiLag2(u,v, u, Δx,Δy,Δt, u_semilag2_coeff)
        adv_v = SemiLag2(u,v, v, Δx,Δy,Δt, u_semilag2_coeff)
 

    if ρ_semilag2_coeff is None:
        ρ = SemiLag(u,v, ρ, Δx,Δy,Δt)
    else:
        ρ = SemiLag2(u,v, ρ, Δx,Δy,Δt, ρ_semilag2_coeff)
    
    ρ = np.minimum(ρ1+0.5, np.maximum(ρ0-0.5, ρ))
    
    ### Mise à jour des conditions limites pour la densité.
    DensityGhostPoints(ρ)
    
    ###### Diffusion step
    ustar = adv_u + Δt*Laplacian(u, Δx,Δy)/Re/ρ
    vstar = adv_v + Δt*Laplacian(v, Δx,Δy)/Re/ρ
    
    ustar += +sin(α) * g * Δt
    vstar += -cos(α) * g * Δt
    
    ###### Mise à jour des conditions limites pour la vitesse :
    VelocityGhostPoints(ustar,vstar)
 
    ### Update divstar 
    divstar = Divergence(ustar,vstar, Δx,Δy)
    divstar /= Δt
    
    if niter%update_side_pressures_modulo == 0:
        phi0_left = np.vectorize(compute_hydrostatic_pressure)(0, Ly-(Ly-y))
        phi0_right = np.vectorize(compute_hydrostatic_pressure)(Lx, Ly-(Ly-y))
        phi0_top = np.vectorize(compute_hydrostatic_pressure)(x, Ly)
    
    ### Solving the linear system
    phi[1:-1, 1:-1] = LAP_solver(RHS=divstar[1:-1,1:-1], BCdir_val_left=phi0_left, BCdir_val_right=phi0_right, BCdir_val_top=phi0_top)
    phi[0   , 1:-1] = 2 * phi0_left  - phi[ 1   , 1:-1]
    phi[  -1, 1:-1] = 2 * phi0_right - phi[-2   , 1:-1]
    phi[1:-1,   -1] = 2 * phi0_top   - phi[ 1:-1,   -2]
    phi[ :  , 0   ] =                + phi[:    ,    1]
    phi[-1,-1] = phi[-1,-2]

    ### Update gradphi
    dx_phi[1:-1, :] = (phi[2:, :] - phi[:-2, :])/Δx/2
    dy_phi[:, 1:-1] = (phi[:, 2:] - phi[:, :-2])/Δy/2

    ### Project u
    u = ustar - Δt*dx_phi / ρ
    v = vstar - Δt*dy_phi / ρ
    
    
    if niter%disp_modulo == 0:
        do_plot(save_image=True)
        np.savez(path_prefix+'/{:04d}.npz'.format(niter//disp_modulo), u=u, v=v, rho=ρ)
    
    t += Δt
    niter += 1

<ipython-input-14-59829d6cde41>:48: RuntimeWarning: overflow encountered in true_divide
  divstar /= Δt
